# Notebook to automate LLM Inference using LangChain
Using Langchain as a framework to interact with Ollama models beyond the Terminal along with the possibility of setting up a pipeline to import an image, run Model inference and export the output to its corresponding study folder

In [1]:
# imports and functions required to convert images into base64 for the model inference
import base64
from io import BytesIO

from IPython.display import HTML, display
from PIL import Image
import pydicom
import numpy as np

In [2]:
# function to convert dcm files to png
def dicom_to_png(dicom_path, png_path):
    ds = pydicom.dcmread(dicom_path)
    pixel_array = ds.pixel_array.astype(float)
    scaled = (np.maximum(pixel_array, 0) / pixel_array.max()) * 255.0
    img = Image.fromarray(scaled.astype(np.uint8)).convert("RGB")
    img.save(png_path)

In [14]:
import os

data_dir = './dataset'

# this gives in each sub folder within dataset s0, s1, s2
folders = sorted(os.listdir(data_dir))
for folder in folders:
   s_folder = os.path.join(data_dir, folder)
   # gives all .dcm and .txt files within each study
   all_files = os.listdir(s_folder)
   dcm_files = [f for f in all_files if f.lower().endswith('.dcm')]
   
   for each_file in dcm_files:
    path_to_save = os.path.join(s_folder, f"{each_file[0:3]}.png")
    dicom_path = os.path.join(s_folder, each_file)
    dicom_to_png(dicom_path, path_to_save)



In [8]:
def convert_to_base64(pil_image):
    """
    Convert PIL images to Base64 encoded strings

    :param pil_image: PIL image
    :return: Re-sized Base64 string
    """

    buffered = BytesIO()
    pil_image.save(buffered, format="PNG")  # You can change the format if needed
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

In [15]:
from langchain_core.messages import HumanMessage
from langchain_ollama import ChatOllama

In [22]:
# function which takes in data and converts into LangChain human message for the Model
def prompt_func(data):
    text = data["text"]
    image = data["image"]

    image_part = {
        "type": "image_url",
        "image_url": f"data:image/jpeg;base64,{image}",
    }

    content_parts = []

    text_part = {"type": "text", "text": text}

    content_parts.append(image_part)
    content_parts.append(text_part)

    return [HumanMessage(content=content_parts)]

In [ ]:
from langchain_core.output_parsers import StrOutputParser
llm = ChatOllama(model="gemma3:4b-it-qat", temperature=0)
chain = prompt_func | llm | StrOutputParser()

In [29]:
data_dir = './dataset'
from PIL import Image

# this gives in each sub folder within dataset s0, s1, s2
folders = sorted(os.listdir(data_dir))

for folder in folders:
    s_folder = os.path.join(data_dir, folder)
    # gives files within each study
    all_files = os.listdir(s_folder)
    image_files = [f for f in all_files if f.lower().endswith('.png')]

    for each_file in image_files:
        image_path = os.path.join(s_folder, each_file)
        # file to write output into
        model_output_path =  os.path.join(s_folder, f"gemma-4b-{each_file[0:3]}.txt")
        img = Image.open(image_path)
        # convert to base64 version of image
        base64_str = convert_to_base64(img)
        query_chain = chain.invoke(
            {"text": "You are a radiologist reviewing this imaging study. Based on the image provided, generate only the *Findings* section of a radiology report. Use structured, concise, and formal language consistent with professional radiology reporting. Do not include the Impression, Conclusion, or Recommendations.", 
            "image": base64_str}
            )
        # write model output into a file
        with open(model_output_path, "w") as file:
            file.write(query_chain)
    